In [3]:
import pandas as pd
import torch

from datetime import datetime
from pprint import pprint

from config import *
from data.extract import DetectorDataProvider, LookUpTable

%reload_ext autoreload
%autoreload 2


### Read data from the data set

Get list of all sections

In [34]:
lut = LookUpTable('Data')
sections = set()
for inter in lut.list_intersections():
    detectors = lut.get_detectors_on(inter)
    for sec in detectors[['Starting Intersection', 'Ending Intersection']].values:
        sections.add(tuple(sorted(sec)))
pprint(list(sections)[:10])

/home/kolusask/TUM/IDP/src/data/extract.py:76: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  self.lookup_table = self.lookup_table[


[('1012', '26&8001'),
 ('2050', '53'),
 ('13', '4140'),
 ('51', '8004'),
 ('3080', 'KO'),
 ('6090', '6100'),
 ('6100', 'NB-South'),
 ('8505', 'Network Boundary'),
 ('5030', '5031'),
 ('43', 'Bussardstrasse')]


Extract list of detectors for each section

In [35]:
lut = LookUpTable('Data')
int_det = []
for int_1, int_2 in sections:
    det_1_2, det_2_1 = lut.get_detectors_between(int_1, int_2)
    int_det.append((int_1, int_2, det_1_2))
    int_det.append((int_2, int_1, det_2_1))
int_det = pd.DataFrame(int_det, columns=['Start', 'End', 'Detectors'])
int_det

/home/kolusask/TUM/IDP/src/data/extract.py:76: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  self.lookup_table = self.lookup_table[


,Start,End,Detectors
0,1012,26&8001,[]
1,26&8001,1012,[3(DB1)]
2,2050,53,[]
3,53,2050,[5(DB2)]
4,13,4140,[]
...,...,...,...
467,8602,33,[]
468,6010,7020,[1(DA1)]
469,7020,6010,[]
470,8602,Network Boundary East,[]


Extract data from January to August and accumulate counts by section

In [36]:
ddp = DetectorDataProvider('Data')

In [1]:
START_DATE = datetime(2021, 1, 1)
END_DATE = datetime(2021, 8, 1)

lens = set()
def get_count(section_end, detectors):
    section_data = ddp.get_data_for_period(section_end, START_DATE, END_DATE)
    for col in section_data.columns:
        section_data[col] = pd.to_numeric(section_data[col], errors='coerce')
    try:
        return list(section_data[detectors].fillna(0).sum(axis=1, numeric_only=True).cumsum())
    except KeyError:
        print(section_end, section_data.columns, detectors)
int_det['Counts'] = int_det.apply(lambda sec: get_count(sec['End'], sec['Detectors']), axis=1)

int_det.to_hdf(out_path('int_det.hdf'), 'int_det')
int_det

NameError: name 'datetime' is not defined

### Write data into a matrix

Write counts into a matrix
$$Q=[q_1,q_2,...,q_p]$$
for
$$q_i=\set{z(s_i,t_1),z(s_i,t_2),...z(s_i,t_d)}^T,$$
$z(s_i,t_j)$ is the traffic flow of the road section $s_i$ within the time interval $(t_0,t_j)$

In [38]:
int_det['Counts'].apply(len)

0      20352
1      20352
2      20352
3      20352
4      20352
       ...  
467    20352
468    20352
469    20352
470    20352
471    20352
Name: Counts, Length: 472, dtype: int64

In [39]:
mat_q = torch.tensor(int_det['Counts'].tolist()).T
torch.save(mat_q, out_path('mat_q.pt'))
mat_q.shape

torch.Size([20352, 472])